In [1]:
pip install langchain
pip install langchain-community
pip install langchain-google-genai
pip install google-generativeai
pip install chromadb
pip install sentence-transformers
pip install pandas
pip install openpyxl

ERROR: Could not find a version that satisfies the requirement tavily (from versions: none)
ERROR: No matching distribution found for tavily


In [2]:

from tavily import TavilyClient
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.embeddings.base import Embeddings
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
import os

In [8]:


class LocalSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        return self.model.encode(text, convert_to_numpy=True).tolist()

client = TavilyClient(api_key="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")
urls = [
    "https://www.itcportal.com/about-itc/shareholder-value/annual-reports/itc-annual-report-2023/pdf/ITC-Report-and-Accounts-2023.pdf",
    "https://www.itcportal.com/about-itc/shareholder-value/annual-reports/itc-annual-report-2024/pdf/ITC-Report-and-Accounts-2024.pdf"
]
response = client.extract(urls=urls, include_images=False, extract_depth="advanced")

docs = []
for r in response["results"]:
    text = r.get("raw_content", "")
    if text:
        docs.append(Document(page_content=text, metadata={"source": r.get("url")}))

excel_path = "/content/aaa.xlsx"
if os.path.exists(excel_path):
    excel_df = pd.read_excel(excel_path)
    for _, row in excel_df.iterrows():
        row_dict = row.astype(str).to_dict()
        row_text = "\n".join([f"{k}: {v}" for k, v in row_dict.items()])
        docs.append(Document(page_content=row_text, metadata={"source": "excel"}))

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

embedding = LocalSentenceTransformerEmbeddings()
vectordb = Chroma.from_documents(split_docs, embedding=embedding)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


KeyboardInterrupt: 

In [14]:
vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory="./chroma_db"
)
vectordb.persist()


<ipython-input-14-6d69f8280909>:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()  # This writes the data to disk


In [5]:
vectordb1 = Chroma(
    embedding_function=embedding,
    persist_directory="./chroma_db"
)
retriever = vectordb.as_retriever()


In [6]:

from google.colab import userdata
import os
import google.generativeai as genai
os.environ["GOOGLE_API_KEY"] = userdata.get('g-key')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0.2)

# Step 6: Create RAG pipeline
retriever = vectordb1.as_retriever()
prompt_template = """
Answer the question using the context below. ".

Context: {context}

Question: {question}
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

rag_chain = (
    {"context": retriever | RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



In [7]:
question = "give price data"
answer = rag_chain.invoke(question)
print("Answer:", answer)

ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/guides/embeddings in query.

In [19]:
!zip -r your_output_name.zip /content/chroma_db


  adding: content/chroma_db/ (stored 0%)
  adding: content/chroma_db/chroma.sqlite3 (deflated 52%)
  adding: content/chroma_db/4774d067-f266-489e-bf0e-8a80fc0d2848/ (stored 0%)
  adding: content/chroma_db/4774d067-f266-489e-bf0e-8a80fc0d2848/index_metadata.pickle (deflated 45%)
  adding: content/chroma_db/4774d067-f266-489e-bf0e-8a80fc0d2848/header.bin (deflated 55%)
  adding: content/chroma_db/4774d067-f266-489e-bf0e-8a80fc0d2848/data_level0.bin (deflated 63%)
  adding: content/chroma_db/4774d067-f266-489e-bf0e-8a80fc0d2848/length.bin (deflated 82%)
  adding: content/chroma_db/4774d067-f266-489e-bf0e-8a80fc0d2848/link_lists.bin (deflated 81%)


In [9]:
!pip freeze > requirements.txt